<a href="https://colab.research.google.com/github/HarryBurgin/TSE-Assignment3/blob/main/TSE_Group_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes** <br>
Would be good if someone could upload the data to colab using kaggle. <br>
I think u can upload data using its URL

**Set up**<br>
Import all relevant libraries

In [59]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

import pandas as pd
from sklearn.model_selection import train_test_split


## Used to read csv data file
from google.colab import files
import csv
import re

**Create data tensors** <br>
Load csv file and clean data

In [60]:
#Function to create a list of reviews and a list of sentiments (+ve/-ve)
def listCreation(inputData, resultData):
    #Opens the csv file
    fileName = open('IMDB Dataset.csv', 'r', encoding='utf-8')
    file = csv.DictReader(fileName)
    #Iterates each row then appends dataset
    for col in file:
        inputData.append(col['review'])
        if col['sentiment'] == 'positive':
          resultData.append(1)
        elif col['sentiment'] == 'negative':
          resultData.append(0)
        else:
          resultData.append("findMe")                          

def dataCleanup(inputData):
    #Iterates through the review
    for i in range(len(inputData)):
      #Replaces any text between <> with a blank
      inputData[i] = re.sub('<.*?>', '', inputData[i])

**Tokenise reviews** <br>
Use of libraries used here is not suggested

In [61]:
## Tokenising the data

# token = Tokenizer(lower = True)
# token.fit_on_texts(x_train)
# x_train = token.texts_to_sequences(x_train)
# x_test = token.texts_to_sequences(x_test)

def tokeniseData(data, Dtype):
  if (Dtype == "train"):
    token.fit_on_texts(data)
  data = token.texts_to_sequences(data)
  return data

**Limiting word count and adding padding**

In [62]:
## Limiting word count and adding padding
def paddingData(data, maxWords):
  for d in range(len(data)): # Loops through data
    if len(data[d]) > maxWords: # If length of data is above max words
      data[d] = data[d][0:120] # Take a slice of the list starting from index 0 to 120

  for d in range(len(data)): # Loop through review
    while len(data[d]) < maxWords: # While the data length is below 120
      data[d].append(0) # Append another 0

  ## Returns the data with padding
  return data

**Pre-processing data**

In [63]:
## Reading data into list
## Splitting data into train and test sets
inputData = []
resultData= []

listCreation(inputData, resultData)
dataCleanup(inputData)


## X is review and y is sent
## Split into train and test data
## random_state set to 0 for testing
x_train, x_test, y_train, y_test = train_test_split(inputData, resultData, test_size=0.2, random_state=0) 
## Could split test data into test and validation

In [64]:
## Tokenising data
## Tokenising been changed because of func
token = Tokenizer(lower = True) ## Token

x_train = tokeniseData(x_train, "train")
x_test = tokeniseData(x_test, "test")  

In [65]:
## Limiting word count and adding padding
maxWords = 120 # Sets max number of words
total_words = len(token.word_index) + 1

x_test = paddingData(x_test, maxWords)
x_train = paddingData(x_train, maxWords)


## Shows that all reviews have same number of words
for _ in x_test:
  if (len(_) != maxWords):
    print(_)

for i in range(len(y_train)):
  if (y_train[i] != 1 and y_train[i] != 0):
    print(i, " ", y_train[i])

In [66]:
## Converting train and test data to tensors
x_train = tf.convert_to_tensor(x_train, dtype=tf.int32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int16)
x_test = tf.convert_to_tensor(x_test, dtype=tf.int32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.int16)

#print(y_train)
#print(x_train)

**Create model**

In [67]:
# model = keras.Sequential()
# model.add(keras.Input(shape=(None, 28)))
# model.add(layers.SimpleRNN(512, return_sequences=True, activation='relu'))
# model.add(layers.SimpleRNN(512, activation='relu'))
# model.add(layers.Dense(10))

EMBED_DIM = 32
LSTM_OUT = 64

model = keras.Sequential()
model.add(layers.Embedding(total_words, EMBED_DIM, input_length = maxWords))
model.add(layers.LSTM(LSTM_OUT, dropout=0.75, recurrent_dropout=0.5))
model.add(layers.Dense(1, activation='sigmoid'))

print(model.summary())


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 120, 32)           3643168   
                                                                 
 lstm_2 (LSTM)               (None, 64)                24832     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3,668,065
Trainable params: 3,668,065
Non-trainable params: 0
_________________________________________________________________
None


**Compile model**

In [68]:
# model.compile(
#     loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     optimizer = keras.optimizers.Adam(learning_rate=0.001),
#     metrics = ["accuracy"]
#     )

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

**Test Zone**

In [69]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, verbose = 1)

Epoch 1/5
313/313 [==============================] - 100s 310ms/step - loss: 0.5188 - accuracy: 0.7355
Epoch 2/5
313/313 [==============================] - 92s 294ms/step - loss: 0.3339 - accuracy: 0.8629
Epoch 3/5
313/313 [==============================] - 92s 295ms/step - loss: 0.2577 - accuracy: 0.8999
Epoch 4/5
313/313 [==============================] - 92s 294ms/step - loss: 0.2130 - accuracy: 0.9191
Epoch 5/5
313/313 [==============================] - 92s 293ms/step - loss: 0.1818 - accuracy: 0.9317


In [70]:
model.evaluate(x_test, y_test, batch_size = 128, verbose = 2)
## Epochs = 5, batch_size = 128
## 0.84 accuracy with no dropout
## 0.86 with dropout=0.75  
## 0.8600 with dropout=0.75, recurrent_dropout=0.5

## Increaced size of test cases

79/79 - 3s - loss: 0.4273 - accuracy: 0.8543 - 3s/epoch - 40ms/step


[0.4272671937942505, 0.8543000221252441]

**Adding predictions**

In [74]:
reviews = []
## Positive 
reviews.append("Fantastic Film")
## Negative
reviews.append("I have no idea who wrote this, but god it feels like it's meant for kids. Not scary, all over the place with irrelevant stories, boring from beginning to ...end? Could not tell, turned it off after the witch flew across town to visit the little girl. Don't care what happens next.")
reviews.append("Yikes. Doctor sleep is a fitting name cuz you're gonna wanna by the time this movies finally over. It feels so much longer than it is. It's corny, cringy, the dialogue is bad, the characters are not believable, the story is predictable and goofy as hell. It's critically just not a good film. But even worse, it's not an interesting or entertaining one.")
reviews.append("Is it me? I thought this Film was so slow and mixed up I was so bored i fell asleep what a waist of time..Stanley Kubrick would be turning in his grave at this one..I should have painted a door and watched it dry that would've been more enjoyable.")
## Poitive
reviews.append("Ok first of all I never read the books I just go to the movies so I never knew until I saw this movie what the shining was and this movie filled in all the gaps that I didn't understand from the first one and I thought it was pretty good it did make me jump a few times in my seat which is really hard to do considering how desensitized I am from seeing so many I don't think people are giving this one enough credit by going to the theater to see it but I'm sure it will do better in video sales.")
reviews.append("I was saving this film aside for quite a while, because I assumed it's just another horror movie (I don't scare easily so I find the horror genre kinda boring), and I finally watched it yesterday... well, the truth is: it's more fantasy than horror, and I was surprised by how brilliant & enjoyable it was!")



reviews = tokeniseData(reviews, "test")
reviews = paddingData(reviews, maxWords)
reviews = tf.convert_to_tensor(reviews, dtype=tf.int32)


model.predict(reviews)

array([[0.99053985],
       [0.00632453],
       [0.01420733],
       [0.01792878],
       [0.9836149 ],
       [0.9818808 ]], dtype=float32)